<a href="https://colab.research.google.com/github/AleksanderDanilian/hacksai_construction_technic/blob/master/ai_hacks_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd 
import numpy as np
import glob
from tqdm import tqdm
from os import listdir
import cv2
from sklearn.model_selection import train_test_split
import os
from collections import Counter

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, WeightedRandomSampler
from torchvision import datasets, models, transforms
from torchvision.models import ResNet50_Weights, Inception_V3_Weights, EfficientNet_B5_Weights, \
EfficientNet_V2_S_Weights, ResNeXt101_64X4D_Weights

from PIL import Image
import matplotlib.pyplot as plt
from IPython.display import clear_output

from sklearn.ensemble import GradientBoostingClassifier
from joblib import dump, load

In [ ]:
# Распаковка архивов с данными

!unzip /content/test_dataset_test.zip 
!unzip /content/train_dataset_train.zip 

In [ ]:
DIR_TRAIN = "/content/train/"
DIR_TEST = "/content/test/"

PATH_TRAIN = "/content/train.csv"
PATH_TEST = "/content/test.csv"

In [ ]:
data_df = pd.read_csv(PATH_TRAIN)

In [ ]:
class ImageDataset(Dataset):
    def __init__(self, data_df, transform=None, test_version=True):

        self.data_df = data_df
        self.transform = transform
        self.test_version = test_version

    def __getitem__(self, idx):
        # достаем имя изображения и его лейбл
        image_name, label = self.data_df.iloc[idx]['ID_img'], self.data_df.iloc[idx]['class']

        # читаем картинку. read the image
        if not self.test_version:
          image = cv2.imread(DIR_TRAIN + f"{image_name}")
        else:
          image = cv2.imread(DIR_TEST + f"{image_name}")
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = Image.fromarray(image)
        
        # преобразуем, если нужно. transform it, if necessary
        if self.transform:
            image = self.transform(image)
        
        return image, torch.tensor(label).long() 
    
    def __len__(self):
        return len(self.data_df)

In [ ]:
class PretrainedModels():

  resnet50_transforms = ResNet50_Weights.DEFAULT.transforms()
  inception_v3_transforms = Inception_V3_Weights.DEFAULT.transforms()
  efficientnet_b5_transforms = EfficientNet_B5_Weights.DEFAULT.transforms()
  resnext_transforms = ResNeXt101_64X4D_Weights.DEFAULT.transforms()

  resnet50_weights = ResNet50_Weights.DEFAULT
  inception_v3_weights = Inception_V3_Weights.DEFAULT
  efficientnet_b5_weights = EfficientNet_B5_Weights.DEFAULT
  resnext_weights = ResNeXt101_64X4D_Weights.DEFAULT

In [ ]:
# задаем преобразование изображения

def build_data_transformers(augmentation = transforms.AutoAugment(), model_transforms = PretrainedModels.resnet50_transforms):
  '''
  Possible auto augmentations transforms.AutoAugment(), transforms.RandAugment(), transforms.TrivialAugmentWide()
  '''

  torch.manual_seed(17)  

  if augmentation:         

    train_transform = transforms.Compose([
        augmentation,
        model_transforms
    ])
  else:
    train_transform = transforms.Compose([
        model_transforms
    ])

  valid_transform = transforms.Compose([
      model_transforms
  ])

  return train_transform, valid_transform


In [ ]:
# разделим датасет на трейн и валидацию, чтобы смотреть на качество

train_df, valid_df = train_test_split(data_df, test_size=0.2, random_state=43, stratify=data_df['class'])


In [ ]:
# функция подготовки генератора для обучния модели

def prepare_loader(model_transforms, model_augmentation, train_df, valid_df, batch_size = 4):
  train_transform, valid_transform = build_data_transformers(augmentation = model_augmentation, 
                                                            model_transforms = model_transforms)
  train_dataset = ImageDataset(train_df, train_transform, False)
  valid_dataset = ImageDataset(valid_df, valid_transform, False)

  # создаем семплер - позволит генератору создавать батчи с равномерным распеределением классов
  num_samples = len(train_dataset)
  labels = train_dataset.data_df['class'].values
  n_classes = len(set(labels))
  cnt = Counter(labels)
  class_weights = [num_samples/cnt[i] for i in range(n_classes)]
  weights = [class_weights[int(labels[i])] for i in range(num_samples)]
  sampler = WeightedRandomSampler(weights, len(train_dataset), replacement=True)

  train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                            batch_size=batch_size,
                                            shuffle=False,
                                            pin_memory=True,
                                            num_workers=2,
                                            sampler=sampler)

  valid_loader = torch.utils.data.DataLoader(dataset=valid_dataset,
                                            batch_size=batch_size,
                                            pin_memory=True,
                                            num_workers=2)
  
  return train_loader, valid_loader

def plot_history(train_history, val_history, title='loss'):
    plt.figure()
    plt.title('{}'.format(title))
    dd = list(map(lambda x: x.cpu().detach().numpy(), train_history))
    plt.plot(dd, label='train', zorder=1)
    
    points = np.array(val_history)
    steps = list(range(0, len(train_history) + 1, int(len(train_history) / len(val_history))))[1:]

    plt.scatter(steps, val_history, marker='+', s=180, c='orange', label='val', zorder=2)
    plt.xlabel('train steps')
    
    plt.legend(loc='best')
    plt.grid()

    plt.show()

def train(model, criterion, optimizer, scheduler, train_dataloader, test_dataloader, NUM_EPOCH=15, model_type = 'inception', cuda = True):
    
    epoch_number = 0
    train_loss_log = []
    val_loss_log = []
    
    train_acc_log = []
    val_acc_log = []

    val_loss_min = 1000 # для сохранения лучших весов модели
    
    for epoch in tqdm(range(NUM_EPOCH)):
        epoch_number +=1
        model.train()
        train_loss = 0.
        train_size = 0
        
        train_pred = 0.

        for imgs, labels in train_dataloader:
            optimizer.zero_grad()
            if cuda:
              imgs = imgs.cuda()
              labels = labels.cuda()

            if model_type == 'inception':
              outputs, aux_outputs = model(imgs)
              loss1 = criterion(outputs, labels)
              loss2 = criterion(aux_outputs, labels)
              loss = loss1 + 0.4*loss2
              loss.backward()     
              train_loss += loss.item()
              train_size += outputs.size(0)
              train_loss_log.append(loss.data / outputs.size(0))
              train_pred += (outputs.argmax(1) == labels).sum()
            
            else:
              y_pred = model(imgs)
              loss = criterion(y_pred, labels)
              # loss.requires_grad = True
              loss.backward() 
              train_loss += loss.item()
              train_size += y_pred.size(0)
              train_loss_log.append(loss.data / y_pred.size(0))
              train_pred += (y_pred.argmax(1) == labels).sum()
            
            optimizer.step()

        train_acc_log.append(train_pred / train_size)

        val_loss = 0.
        val_size = 0
        
        val_pred = 0.
        
        model.eval()
        
        with torch.no_grad():
            for imgs, labels in test_dataloader:
                if cuda:
                  imgs = imgs.cuda()
                  labels = labels.cuda()
                
                pred = model(imgs)
                loss = criterion(pred, labels)
                
                val_loss += loss.item()
                val_size += pred.size(0)
                
                val_pred += (pred.argmax(1) == labels).sum()

        if val_loss < val_loss_min:
          val_loss_min = val_loss
          print(val_loss)
          torch.save(model.state_dict(), f'/content/{model_type}_model.pth')
          print('model_saved. val_loss = ', val_loss_min)
          print('epoch number ', epoch_number)
        
        # Update LR
        scheduler.step(val_loss)          
        lr_step = optimizer.state_dict()["param_groups"][0]["lr"]       

        val_loss_log.append(val_loss / val_size)
        val_acc_log.append(val_pred / val_size)

        # clear_output()
        plot_history(train_loss_log, val_loss_log, 'loss')
        
        print('lr_step', lr_step)
        print('Train loss:', (train_loss / train_size)*100)
        print('Val loss:', (val_loss / val_size)*100)
        print('Train acc:', (train_pred / train_size)*100)
        print('Val acc:', (val_pred / val_size)*100)
        
    return train_loss_log, train_acc_log, val_loss_log, val_acc_log
  
def get_preds(model, loader):
  with torch.no_grad():

    all_answers = np.array([])

    for imgs, labels in loader:

      imgs = imgs.cuda() #
      labels = labels.cuda()
      
      batch_preds = model(imgs).squeeze(1).softmax(1) # get probabilities
      batch_preds = batch_preds.cpu().numpy()

      if all_answers.shape != (0,):
        all_answers = np.vstack((all_answers, batch_preds))
      else:
        all_answers = batch_preds

  print(all_answers.shape)
  
  return all_answers 

In [ ]:
def train_model(train_df, valid_df, batch_size=16, NUM_EPOCH=25, model_type='inception'):

  if model_type == 'efficientnet':
    model = models.efficientnet_b5(weights = PretrainedModels.efficientnet_b5_weights)
    model.classifier[1] = nn.Linear(2048, 8) 
    train_loader, valid_loader = prepare_loader(PretrainedModels.efficientnet_b5_transforms, transforms.AutoAugment(), train_df, valid_df, batch_size)
  elif model_type == 'inception':
    model = models.inception_v3(weights = PretrainedModels.inception_v3_weights)
    model.aux_logits=True
    model.AuxLogits.fc = nn.Linear(768, 8)
    model.fc = nn.Linear(2048, 8)
    train_loader, valid_loader = prepare_loader(PretrainedModels.inception_v3_transforms, transforms.AutoAugment(), train_df, valid_df, batch_size)
  elif model_type == 'resnet':
    model = models.resnet50(weights=PretrainedModels.resnet50_weights)
    model.fc = nn.Linear(2048, 8)
    train_loader, valid_loader = prepare_loader(PretrainedModels.resnet50_transforms, transforms.RandAugment(), train_df, valid_df, batch_size)
  elif model_type == 'resnext':
    model = models.resnext101_64x4d(weights = PretrainedModels.resnext_weights)
    model.fc = nn.Linear(2048, 8)
    train_loader, valid_loader = prepare_loader(PretrainedModels.resnext_transforms, transforms.TrivialAugmentWide(), train_df, valid_df, batch_size)
  else:
    print('no such model is available')
  
  model = model.cuda()

  criterion = torch.nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
  scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.1, threshold = 0.1, patience=6, verbose = True)
  train_loss_log, train_acc_log, val_loss_log, val_acc_log = train(model, criterion, optimizer, scheduler, train_loader, valid_loader, NUM_EPOCH, model_type, cuda = True)

  return model, valid_loader


In [ ]:
# RESNET
resnet_model, resnet_valid_loader = train_model(train_df, valid_df, batch_size=16, NUM_EPOCH=25, model_type='resnet')
resnet_model.eval()

# RESNEXT
resnext_model, resnext_valid_loader = train_model(train_df, valid_df, batch_size=16, NUM_EPOCH=25, model_type='resnext')
resnext_model.eval()

# EFFICIENTNET_b5
efficientnet_model, efficientnet_valid_loader = train_model(train_df, valid_df, batch_size=16, NUM_EPOCH=25, model_type='efficientnet')
efficientnet_model.eval()

# INCEPTION_v3
inception_model, inception_valid_loader = efficientnet_model = train_model(train_df, valid_df, batch_size=16, NUM_EPOCH=25, model_type='inception')
inception_model.eval()

In [ ]:
resnet_model

In [ ]:
# создаем даталоадеры для каждой модели отдельно (т.к. обучаем xgboost classifier - берем валидационные данные, которые наши нейронки не видели.)

efficientnet_preds = get_preds(efficientnet_model[0], efficientnet_valid_loader)
resnet_preds = get_preds(resnet_model, resnet_valid_loader)
inception_preds = get_preds(inception_model, inception_valid_loader)
resnext_preds = get_preds(resnext_model, resnext_valid_loader)

all_models_preds = np.concatenate((resnet_preds, inception_preds, resnext_preds, efficientnet_preds), 1)
valid_labels = valid_df['class'].values.astype(int)

(998, 8)
(998, 8)
(998, 8)
(998, 8)


In [ ]:
X_train, X_test = all_models_preds[:700], all_models_preds[700:]
y_train, y_test = valid_labels[:700], valid_labels[700:]

clf = GradientBoostingClassifier(n_estimators=2500, learning_rate=0.01, max_depth=1, random_state=0, subsample = 1).fit(X_train, y_train)
print(clf.score(X_test, y_test))

dump(clf, '/content/xgboost_model.joblib') 

0.785234899328859


['/content/xgboost_model.joblib']

In [ ]:
# проверяем наш xgboost классификатор на тестовом наборе данных

test_df = pd.read_csv(PATH_TEST)

resnet_dataset = ImageDataset(test_df, PretrainedModels.resnet50_transforms, True)
resnet_loader = torch.utils.data.DataLoader(dataset=resnet_dataset,
                                           batch_size=16,
                                           pin_memory=True,
                                           num_workers=2)

inception_dataset = ImageDataset(test_df, PretrainedModels.inception_v3_transforms, True)
inception_loader = torch.utils.data.DataLoader(dataset=inception_dataset,
                                           batch_size=16,
                                           pin_memory=True,
                                           num_workers=2)

resnext_dataset = ImageDataset(test_df, PretrainedModels.resnext_transforms, True)
resnext_loader = torch.utils.data.DataLoader(dataset=resnext_dataset,
                                           batch_size=16,
                                           pin_memory=True,
                                           num_workers=2)

efficientnet_dataset = ImageDataset(test_df, PretrainedModels.efficientnet_b5_transforms, True)
efficientnet_loader = torch.utils.data.DataLoader(dataset=efficientnet_dataset,
                                           batch_size=16,
                                           pin_memory=True,
                                           num_workers=2)


efficientnet_test_preds = get_preds(efficientnet_model[0], efficientnet_loader)
resnet_test_preds = get_preds(resnet_model, resnet_loader)
inception_test_preds = get_preds(inception_model, inception_loader)
resnext_test_preds = get_preds(resnext_model, resnext_loader)

all_models_test_preds = np.concatenate((resnet_test_preds, inception_test_preds, resnext_test_preds, efficientnet_test_preds), 1)

xgboost_predicts = clf.predict(all_models_test_preds)
test_df = test_df.drop(["class"], axis = 1)
test_df["class"] = xgboost_predicts

test_df.to_csv("/content/submit.csv", index=False)

(2138, 8)
(2138, 8)
